In [1]:
import os
import wfdb as wf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from biosppy.signals import ecg
from glob import glob

In [2]:
def get_records():
    """ Get paths for data in data/mit/ directory """
    # There are 3 files for each record
    # *.atr is one of them
    path_source = '../MIT-BIH//*.atr'
    paths = glob(path_source)

    # Get rid of the extension
    paths = [path[:-4] for path in paths]
    paths.sort()

    return paths

In [3]:
records = get_records()
print('Total files: ', len(records))
realbeats = ['N','L','R','B','A','a','J','S','V','r',
             'F','e','j','n','E','/','f','Q','?']
normal=['N','L','R','B','e','j'] #0(N)
supraventricular=['A','J','S','a','n'] #1(S)
ventricular=['V','E','r'] #2(V)
fusion= ['F'] #3(F)
unknownbeats=['/','f','Q','?'] #4(Q)


Total files:  5


In [4]:
for path in records:
    pathpts = path.split('/')
    fn = pathpts[-1]
    print('Loading file:', path)

    # Read in the data
    record = wf.rdsamp(path)
    annotation = wf.rdann(path, 'atr')

    # Print some meta informations
    print('    Sampling frequency used for this record:', record[1].get('fs'))
    print('    Shape of loaded data array:', record[0].shape)
    print('    Number of loaded annotations:', len(annotation.num))
    
    # Get the ECG values from the file.
    data = record[0].transpose()

    # Generate the classifications based on the annotations.
    # 0.0 = undetermined
    # 1.0 = normal
    # 2.0 = abnormal
    cat = np.array(annotation.symbol)
    rate = np.zeros_like(cat, dtype='float')
    for catid, catval in enumerate(cat):
        if (catval in normal):
            rate[catid] = 1.0 # Normal
        elif (catval in supraventricular):
            rate[catid] = 2.0 # supraventricular
        elif (catval in ventricular):
            rate[catid] = 3.0 # ventricular
        elif (catval in fusion):
            rate[catid] = 4.0 # ventricular
        elif (catval in unknownbeats):
            rate[catid] = 5.0 # ventricular
    rates = np.zeros_like(data[0], dtype='float')
    rates[annotation.sample] = rate
    
    indices = np.arange(data[0].size, dtype='int')

    # Process each channel separately (2 per input file).
    for channelid, channel in enumerate(data):
        chname = record[1].get('sig_name')[channelid]
        print('     ECG channel type:', chname)
        
        # Find rpeaks in the ECG data. Most should match with
        # the annotations.
        out = ecg.ecg(signal=channel, sampling_rate=360, show=False)
        rpeaks = np.zeros_like(channel, dtype='float')
        rpeaks[out['rpeaks']] = 1.0
        
        beatstoremove = np.array([0])

        # Split into individual heartbeats. For each heartbeat
        # record, append classification (normal/abnormal).
        beats = np.split(channel, out['rpeaks'])
        for idx, idxval in enumerate(out['rpeaks']):
            firstround = idx == 0
            lastround = idx == len(beats) - 1

            # Skip first and last beat.
            if (firstround or lastround):
                continue

            # Get the classification value that is on
            # or near the position of the rpeak index.
            fromidx = 0 if idxval < 10 else idxval - 10
            toidx = idxval + 10
            catval = rates[fromidx:toidx].max()
            
            # Skip beat if there is no classification.
            if (catval == 0.0):
                beatstoremove = np.append(beatstoremove, idx)
                continue

            # Normal beat is now classified as 0.0 and abnormal is 1.0.
            catval = catval - 1.0

            # Append some extra readings from next beat.
            beats[idx] = np.append(beats[idx], beats[idx+1][:40])

            # Normalize the readings to a 0-1 range for ML purposes.
            beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()

            # Resample from 360Hz to 125Hz
            newsize = int((beats[idx].size * 125 / 360) + 0.5)
            beats[idx] = signal.resample(beats[idx], newsize)

            # Skipping records that are too long.
            if (beats[idx].size > 187):
                beatstoremove = np.append(beatstoremove, idx)
                continue

            # Pad with zeroes.
            zerocount = 187 - beats[idx].size
            beats[idx] = np.pad(beats[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))

            # Append the classification to the beat data.
            beats[idx] = np.append(beats[idx], catval)

        beatstoremove = np.append(beatstoremove, len(beats)-1)

        # Remove first and last beats and the ones without classification.
        beats = np.delete(beats, beatstoremove)

        # Save to CSV   file.
        savedata = np.array(list(beats[:]), dtype=np.float)
        outfn = '../File CSV\\'+fn+'_'+chname+'.csv'
        print('    Generating ', outfn)
        with open(outfn, "wb") as fin:
            np.savetxt(fin, savedata, delimiter=",", fmt='%f')

Loading file: ../NSR\19088
    Sampling frequency used for this record: 128
    Shape of loaded data array: (10967040, 2)
    Number of loaded annotations: 117880
     ECG channel type: ECG1


c:\Users\ASUS\anaconda3\envs\KTDL\lib\site-packages\numpy\lib\function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17300\2098647787.py:106: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  savedata = np.array(list(beats[:]), dtype=np.float)


    Generating  ../File CSV\NSR\19088_ECG1.csv
     ECG channel type: ECG2
    Generating  ../File CSV\NSR\19088_ECG2.csv
Loading file: ../NSR\19090
    Sampling frequency used for this record: 128
    Shape of loaded data array: (11143168, 2)
    Number of loaded annotations: 81953
     ECG channel type: ECG1


c:\Users\ASUS\anaconda3\envs\KTDL\lib\site-packages\numpy\lib\function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17300\2098647787.py:106: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  savedata = np.array(list(beats[:]), dtype=np.float)


    Generating  ../File CSV\NSR\19090_ECG1.csv
     ECG channel type: ECG2
    Generating  ../File CSV\NSR\19090_ECG2.csv
Loading file: ../NSR\19093
    Sampling frequency used for this record: 128
    Shape of loaded data array: (10705920, 2)
    Number of loaded annotations: 83670
     ECG channel type: ECG1


...

    Generating  ../File CSV\NSR\19093_ECG1.csv
     ECG channel type: ECG2
    Generating  ../File CSV\NSR\19093_ECG2.csv
Loading file: ../NSR\19140
    Sampling frequency used for this record: 128
    Shape of loaded data array: (11139072, 2)
    Number of loaded annotations: 96992
     ECG channel type: ECG1


c:\Users\ASUS\anaconda3\envs\KTDL\lib\site-packages\numpy\lib\function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17300\2098647787.py:106: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  savedata = np.array(list(beats[:]), dtype=np.float)


    Generating  ../File CSV\NSR\19140_ECG1.csv
     ECG channel type: ECG2
    Generating  ../File CSV\NSR\19140_ECG2.csv
Loading file: ../NSR\19830
    Sampling frequency used for this record: 128
    Shape of loaded data array: (10701824, 2)
    Number of loaded annotations: 111263
     ECG channel type: ECG1


c:\Users\ASUS\anaconda3\envs\KTDL\lib\site-packages\numpy\lib\function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17300\2098647787.py:106: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  savedata = np.array(list(beats[:]), dtype=np.float)


    Generating  ../File CSV\NSR\19830_ECG1.csv
     ECG channel type: ECG2
    Generating  ../File CSV\NSR\19830_ECG2.csv
